In [1]:
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber 
from nipype.interfaces.utility import IdentityInterface, Function    
from nipype.pipeline.engine import Node, Workflow, JoinNode, MapNode
from nipype.interfaces import fsl
from nipype.interfaces import brainsuite
from nipype.interfaces.io import SelectFiles, DataSink, DataGrabber
from shutil import copyfile
from pandas import Series
from glob import glob
from os.path import abspath, expanduser, join
from os import chdir, remove, getcwd
from nipype import config, logging
from datetime import date
today = str(date.today())
config.enable_debug_mode()

In [3]:
#Set variables
# subject_list = ['02-T2', '06-T1', '07-T2','08-T1', '09-T2', '10-T1', '11-T1', '12-T1', '13-T1', '15-T1', '16-T1', '17-T1',
#                 '18-T1', '19-T1', '20-T1', '21-T1', '22-T1', '23-T1', '24-T1', '29-T1', '30-T1', '31-T1', '32-T1',
#                 '35-T1', '36-T1', '37-T1', '38-T1', '40-T1', '102-T1', '103-T1', '104-T1', '105-T1', '106-T1', '107-T1', '108-T1', '109-T1',
#                 '110-T1', '111-T1']

subject_list = ['29-T1','31-T1'] #'13-T1', '20-T1','22-T1',

home = '/Volumes/group/proc/TIGERanalysis/qT1'
fpath = join(home, 'subjDir')
workflow_dir = join(home, 'workflow_qT1')
data_dir = join(home, 'DataDir_qT1')

In [4]:
#Setup Datasink, Infosource, Selectfiles

datasink = Node(DataSink(base_directory = data_dir),
                   name='datasink')

#Set infosource iterables
infosource = Node(IdentityInterface(fields=['subject_id']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]

#SelectFiles
template = dict(qt1=fpath+'/{subject_id}/man_reg/man_reg_qt1.nii.gz',
               unc_maskL = fpath+'/{subject_id}/Left_Uncinate.nii.gz',
               unc_maskR = fpath+'/{subject_id}/Right_Uncinate.nii.gz',
               cc_maskL = fpath+'/{subject_id}/Left_Cingulum_Cingulate.nii.gz',
               cc_maskR = fpath+'/{subject_id}/Right_Cingulum_Cingulate.nii.gz')

# 
#             bval=fpath+'/{subject_id}/dti.bval',
#             bvec=fpath+'/{subject_id}/dti.bvec'

sf = Node(SelectFiles(template, 
                      base_directory = fpath),
          name = 'sf')


In [5]:
#Extract voxel values from qT1 beneath overlaid mask - Left Uncinate
get_voxLUF = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_LeftUncinate.txt'),
               name = 'get_voxLUF')

In [6]:
#Extract voxel values from qT1 beneath overlaid mask - Left Uncinate
get_voxLCC = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_LeftCC.txt'),
               name = 'get_voxLCC')

In [7]:
# #Extract voxel values from qT1 beneath overlaid mask - Left dAcc MRS Voxel
# get_voxLMRS = Node(fsl.ImageMeants(show_all = True,
#                                out_file = 'ROI_voxel-contrasts_LeftdACC_Voxel.txt'),
#                name = 'get_voxLMRS')

In [8]:
#Extract voxel values from qT1 beneath overlaid mask - Right Uncinate
get_voxRUF = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_RightUncinate.txt'),
               name = 'get_voxRUF')

In [9]:
#Extract voxel values from qT1 beneath overlaid mask - Right Uncinate
get_voxRCC = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_RightCC.txt'),
               name = 'get_voxRCC')

In [11]:
get_stats_manReg=Workflow(name='get_stats_manReg')
get_stats_manReg.connect([(infosource, sf, [('subject_id','subject_id')]),
                   (sf, get_voxLUF, [('qt1', 'in_file')]),
                   (sf, get_voxLUF, [('unc_maskL', 'mask')]),
                   (get_voxLUF, datasink, [('out_file', '3_ManualReg_Stats')]),
                   (sf, get_voxLCC, [('qt1', 'in_file')]),
                   (sf, get_voxLCC, [('cc_maskL', 'mask')]),
                   (get_voxLCC, datasink, [('out_file', '3_ManualReg_Stats.@par')]),
                   (sf, get_voxRUF, [('qt1', 'in_file')]),
                   (sf, get_voxRUF, [('unc_maskR', 'mask')]),
                   (get_voxRUF, datasink, [('out_file', '3_ManualReg_Stats.@par.@par')]),
                   (sf, get_voxRCC, [('qt1', 'in_file')]),
                   (sf, get_voxRCC, [('cc_maskR', 'mask')]),
                   (get_voxRCC, datasink, [('out_file', '3_ManualReg_Stats.@par.@par.@par')])
                  ])
get_stats_manReg.base_dir = workflow_dir
get_stats_manReg.write_graph(graph2use = 'flat')
preproc = get_stats_manReg.run('MultiProc', plugin_args={'n_procs': 4})

190330-20:38:51,949 nipype.workflow DEBUG:
	 (get_stats.infosource, get_stats.sf): No edge data
190330-20:38:51,950 nipype.workflow DEBUG:
	 (get_stats.infosource, get_stats.sf): new edge data: {'connect': [('subject_id', 'subject_id')]}
190330-20:38:51,951 nipype.workflow DEBUG:
	 (get_stats.sf, get_stats.get_voxLUF): No edge data
190330-20:38:51,952 nipype.workflow DEBUG:
	 (get_stats.sf, get_stats.get_voxLUF): new edge data: {'connect': [('qt1', 'in_file')]}
190330-20:38:51,953 nipype.workflow DEBUG:
	 (get_stats.sf, get_stats.get_voxLUF): Edge data exists: {'connect': [('qt1', 'in_file')]}
190330-20:38:51,954 nipype.workflow DEBUG:
	 (get_stats.sf, get_stats.get_voxLUF): new edge data: {'connect': [('qt1', 'in_file'), ('unc_maskL', 'mask')]}
190330-20:38:51,954 nipype.workflow DEBUG:
	 (get_stats.get_voxLUF, get_stats.datasink): No edge data
190330-20:38:51,955 nipype.workflow DEBUG:
	 (get_stats.get_voxLUF, get_stats.datasink): new edge data: {'connect': [('out_file', '3_ManualReg

190330-20:39:06,931 nipype.workflow DEBUG:
	 [Node] No hashfiles found in "/Volumes/group/proc/TIGERanalysis/qT1/workflow_qT1/get_stats/_subject_id_31-T1/sf".
190330-20:39:06,932 nipype.workflow DEBUG:
	 Checking hash "get_stats.sf" locally: cached=False, updated=False.
190330-20:39:06,950 nipype.workflow DEBUG:
	 [MultiProc] Submitted task get_stats.sf (taskid=1).
190330-20:39:06,953 nipype.workflow DEBUG:
	 Allocating get_stats.sf ID=6 (0.20GB, 1 threads). Free: 28.40GB, 2 threads.
190330-20:39:06,954 nipype.workflow DEBUG:
	 Setting node inputs
190330-20:39:06,954 nipype.workflow INFO:
	 [Node] Setting-up "get_stats.sf" in "/Volumes/group/proc/TIGERanalysis/qT1/workflow_qT1/get_stats/_subject_id_31-T1/sf".
190330-20:39:07,13 nipype.workflow DEBUG:
	 [Node] Hashes: [('base_directory', '/Volumes/group/proc/TIGERanalysis/qT1/subjDir'), ('force_lists', False), ('raise_on_empty', True), ('sort_filelist', True), ('subject_id', '31-T1')], 9b8345cdf8335e51d1fe9b9daa6cf6d8, /Volumes/group/pr

190330-20:39:28,622 nipype.workflow DEBUG:
	 output: qt1
190330-20:39:28,623 nipype.workflow DEBUG:
	 [Node] get_voxRCC - setting input in_file = /Volumes/group/proc/TIGERanalysis/qT1/subjDir/31-T1/man_reg/man_reg_qt1.nii.gz
190330-20:39:28,623 nipype.workflow DEBUG:
	 input: mask
190330-20:39:28,624 nipype.workflow DEBUG:
	 results file: /Volumes/group/proc/TIGERanalysis/qT1/workflow_qT1/get_stats/_subject_id_31-T1/sf/result_sf.pklz
190330-20:39:28,625 nipype.utils DEBUG:
	 Loading pkl: /Volumes/group/proc/TIGERanalysis/qT1/workflow_qT1/get_stats/_subject_id_31-T1/sf/result_sf.pklz
190330-20:39:28,997 nipype.workflow DEBUG:
	 output: cc_maskR
190330-20:39:28,998 nipype.workflow DEBUG:
	 [Node] get_voxRCC - setting input mask = /Volumes/group/proc/TIGERanalysis/qT1/subjDir/31-T1/Right_Cingulum_Cingulate.nii.gz
190330-20:39:29,247 nipype.workflow DEBUG:
	 [Node] Directory not found "/Volumes/group/proc/TIGERanalysis/qT1/workflow_qT1/get_stats/_subject_id_31-T1/get_voxRCC".
190330-20:39:

190330-20:39:44,902 nipype.workflow DEBUG:
	 Cannot allocate job 8 (0.20GB, 1 threads).
190330-20:39:44,903 nipype.workflow DEBUG:
	 Cannot allocate job 9 (0.20GB, 1 threads).
190330-20:39:44,903 nipype.workflow DEBUG:
	 Cannot allocate job 10 (0.20GB, 1 threads).
190330-20:39:44,905 nipype.workflow DEBUG:
	 Progress: 12 jobs, 2/4/4 (done/running/ready), 4/6 (pending_tasks/waiting).
190330-20:39:44,905 nipype.workflow DEBUG:
	 Tasks currently running: 4. Pending: 4.
190330-20:39:44,907 nipype.workflow INFO:
	 [MultiProc] Running 4 tasks, and 4 jobs ready. Free memory (GB): 28.00/28.80, Free processors: 0/4.
                     Currently running:
                       * get_stats.get_voxLUF
                       * get_stats.get_voxLCC
                       * get_stats.get_voxRUF
                       * get_stats.get_voxRCC
190330-20:39:44,907 nipype.workflow DEBUG:
	 No resources available
190330-20:39:30,853 nipype.workflow INFO:
	 [Node] Setting-up "get_stats.get_voxLCC" in "/Vol

	 out_file_ROI_voxel-contrasts_LeftCC.txt

190330-20:40:03,557 nipype.interface DEBUG:
	 show_all_True
190330-20:40:03,866 nipype.workflow INFO:
	 [Node] Running "get_voxLUF" ("nipype.interfaces.fsl.utils.ImageMeants"), a CommandLine Interface with command:
fslmeants -i /Volumes/group/proc/TIGERanalysis/qT1/subjDir/31-T1/man_reg/man_reg_qt1.nii.gz -m /Volumes/group/proc/TIGERanalysis/qT1/subjDir/31-T1/Left_Uncinate.nii.gz --order=1 -o ROI_voxel-contrasts_LeftUncinate.txt --showall
190330-20:40:03,947 nipype.interface DEBUG:
	 in_file_/Volumes/group/proc/TIGERanalysis/qT1/subjDir/31-T1/man_reg/man_reg_qt1.nii.gz
190330-20:40:03,951 nipype.interface DEBUG:
	 mask_/Volumes/group/proc/TIGERanalysis/qT1/subjDir/31-T1/Left_Uncinate.nii.gz
190330-20:40:03,955 nipype.interface DEBUG:
	 order_1
190330-20:40:03,958 nipype.interface DEBUG:
	 out_file_ROI_voxel-contrasts_LeftUncinate.txt
190330-20:40:03,961 nipype.interface DEBUG:
	 show_all_True
190330-20:40:04,254 nipype.workflow DEBUG:
	 Needed

...

190330-20:41:04,229 nipype.workflow DEBUG:
	 output: unc_maskL
190330-20:41:04,230 nipype.workflow DEBUG:
	 [Node] get_voxLUF - setting input mask = /Volumes/group/proc/TIGERanalysis/qT1/subjDir/29-T1/Left_Uncinate.nii.gz
190330-20:41:01,973 nipype.workflow DEBUG:
	 [Node] Directory not found "/Volumes/group/proc/TIGERanalysis/qT1/workflow_qT1/get_stats/_subject_id_31-T1/datasink".190330-20:41:02,280 nipype.workflow DEBUG:
	 [Node] Writing post-exec report to "/Volumes/group/proc/TIGERanalysis/qT1/workflow_qT1/get_stats/_subject_id_29-T1/get_voxRUF/_report/report.rst"

190330-20:41:04,270 nipype.utils DEBUG:
	 Removing contents of /Volumes/group/proc/TIGERanalysis/qT1/workflow_qT1/get_stats/_subject_id_31-T1/datasink
190330-20:41:04,275 nipype.utils DEBUG:
	 Creating directory /Volumes/group/proc/TIGERanalysis/qT1/workflow_qT1/get_stats/_subject_id_31-T1/datasink
190330-20:41:04,285 nipype.workflow DEBUG:
	 [Node] Directory not found "/Volumes/group/proc/TIGERanalysis/qT1/workflow_qT1/

	 Linking File: /Volumes/group/proc/TIGERanalysis/qT1/DataDir_qT1/3_ManualReg_Stats/_subject_id_31-T1/ROI_voxel-contrasts_LeftUncinate.txt->/Volumes/group/proc/TIGERanalysis/qT1/workflow_qT1/get_stats/_subject_id_31-T1/get_voxLUF/ROI_voxel-contrasts_LeftUncinate.txt
190330-20:41:20,297 nipype.workflow INFO:
	 [Job 9] Completed (get_stats.get_voxLCC).
190330-20:41:20,299 nipype.workflow DEBUG:
	 Tasks currently running: 2. Pending: 2.
190330-20:41:20,301 nipype.workflow INFO:
	 [MultiProc] Running 2 tasks, and 0 jobs ready. Free memory (GB): 28.40/28.80, Free processors: 2/4.
                     Currently running:
                       * get_stats.get_voxLUF
                       * get_stats.datasink
190330-20:41:20,674 nipype.utils DEBUG:
	 Copying File: /Volumes/group/proc/TIGERanalysis/qT1/DataDir_qT1/3_ManualReg_Stats/_subject_id_31-T1/ROI_voxel-contrasts_LeftUncinate.txt->/Volumes/group/proc/TIGERanalysis/qT1/workflow_qT1/get_stats/_subject_id_31-T1/get_voxLUF/ROI_voxel-contrast

190330-20:41:27,79 nipype.workflow DEBUG:
	 output: out_file
190330-20:41:27,81 nipype.workflow DEBUG:
	 [Node] datasink - setting input 3_ManualReg_Stats.@par = /Volumes/group/proc/TIGERanalysis/qT1/workflow_qT1/get_stats/_subject_id_29-T1/get_voxLCC/ROI_voxel-contrasts_LeftCC.txt
190330-20:41:27,82 nipype.workflow DEBUG:
	 input: 3_ManualReg_Stats.@par.@par
190330-20:41:27,83 nipype.workflow DEBUG:
	 results file: /Volumes/group/proc/TIGERanalysis/qT1/workflow_qT1/get_stats/_subject_id_29-T1/get_voxRUF/result_get_voxRUF.pklz
190330-20:41:27,83 nipype.utils DEBUG:
	 Loading pkl: /Volumes/group/proc/TIGERanalysis/qT1/workflow_qT1/get_stats/_subject_id_29-T1/get_voxRUF/result_get_voxRUF.pklz
190330-20:41:27,148 nipype.utils DEBUG:
	 File: /Volumes/group/proc/TIGERanalysis/qT1/DataDir_qT1/3_ManualReg_Stats/_subject_id_31-T1/ROI_voxel-contrasts_RightUncinate.txt already exists,673959f4fae53cb71c1cfad13ef77311, copy:1
190330-20:41:27,182 nipype.workflow DEBUG:
	 output: out_file
190330-20:

	 Needed files: /Volumes/group/proc/TIGERanalysis/qT1/DataDir_qT1/3_ManualReg_Stats/_subject_id_31-T1/ROI_voxel-contrasts_LeftUncinate.txt;/Volumes/group/proc/TIGERanalysis/qT1/DataDir_qT1/3_ManualReg_Stats/_subject_id_31-T1/ROI_voxel-contrasts_LeftCC.txt;/Volumes/group/proc/TIGERanalysis/qT1/DataDir_qT1/3_ManualReg_Stats/_subject_id_31-T1/ROI_voxel-contrasts_RightUncinate.txt;/Volumes/group/proc/TIGERanalysis/qT1/DataDir_qT1/3_ManualReg_Stats/_subject_id_31-T1/ROI_voxel-contrasts_RightCC.txt;/Volumes/group/proc/TIGERanalysis/qT1/workflow_qT1/get_stats/_subject_id_31-T1/get_voxLUF/ROI_voxel-contrasts_LeftUncinate.txt;/Volumes/group/proc/TIGERanalysis/qT1/workflow_qT1/get_stats/_subject_id_31-T1/get_voxLCC/ROI_voxel-contrasts_LeftCC.txt;/Volumes/group/proc/TIGERanalysis/qT1/workflow_qT1/get_stats/_subject_id_31-T1/get_voxRUF/ROI_voxel-contrasts_RightUncinate.txt;/Volumes/group/proc/TIGERanalysis/qT1/workflow_qT1/get_stats/_subject_id_31-T1/get_voxRCC/ROI_voxel-contrasts_RightCC.txt;/Vol

	 Copying File: /Volumes/group/proc/TIGERanalysis/qT1/DataDir_qT1/3_ManualReg_Stats/_subject_id_29-T1/ROI_voxel-contrasts_RightCC.txt->/Volumes/group/proc/TIGERanalysis/qT1/workflow_qT1/get_stats/_subject_id_29-T1/get_voxRCC/ROI_voxel-contrasts_RightCC.txt
190330-20:41:48,513 nipype.utils DEBUG:
	 /Volumes/group/proc/TIGERanalysis/qT1/DataDir_qT1/3_ManualReg_Stats/_subject_id_29-T1/ROI_voxel-contrasts_RightCC.txt
190330-20:41:48,989 nipype.utils DEBUG:
	 File: /Volumes/group/proc/TIGERanalysis/qT1/DataDir_qT1/3_ManualReg_Stats/_subject_id_29-T1/ROI_voxel-contrasts_RightCC.txt already exists,bae3d83861c3f2cdfe9e743e64d9bc46, copy:1
190330-20:41:49,167 nipype.utils DEBUG:
	 File: /Volumes/group/proc/TIGERanalysis/qT1/DataDir_qT1/3_ManualReg_Stats/_subject_id_29-T1/ROI_voxel-contrasts_RightCC.txt already exists, not overwriting, copy:1
190330-20:41:49,670 nipype.workflow DEBUG:
	 Needed files: /Volumes/group/proc/TIGERanalysis/qT1/DataDir_qT1/3_ManualReg_Stats/_subject_id_29-T1/ROI_voxel-